# A Test for the Datashader Application

This uses [importnb](https://pypi.org/project/importnb) to run the
[Datashader_Example](./Datashader_Example.ipynb) notebook.

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from ipyradiant import LoadWidget
from rdflib import Graph
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from Datashader_Example import ds_example, W
    except ImportError:
        from .Datashader_Example import ds_example, W

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Example data is loaded from [schema.org](https://schema.org/docs/developers.html).

In [ ]:
TEST_URL = "https://raw.githubusercontent.com/schemaorg/schemaorg/v9.0-release/data/releases/8.0/schema.{fmt}"

Tests are implemented as buttons you can click.

In [ ]:
loader = LoadWidget()

In [ ]:
tests = [W.Button(description=f"{fmt}") for fmt in ["ttl", "nt", "rdf"]]
ds_example.layout.width = "100%"

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
def _run_test(btn):
    session = CachedSession(".cache")
    fmt = btn.description

    try:
        timestamp(fmt, "starting...")
        ds_example.graph = Graph()
        timestamp(fmt, "cleaned...")
        assert ds_example.ds.output_graph == None
        url = TEST_URL.format(fmt=fmt)
        timestamp(fmt, f"fetching {url}...")
        response = session.get(url)
        timestamp(fmt, f"... response: {response.status_code}")
        assert response.status_code == 200
        timestamp(fmt, f"... {len(response.text)} bytes")
        timestamp(fmt, f"loading...")
        loader.load_box.file_upload_value = {
            Path(url).name: {
                "metadata": {"length": len(response.text)},
                "content": response.text,
            }
        }
        timestamp(fmt, f"... {len(loader.graph)} triples loaded")
        ds_example.graph = loader.graph
        assert len(ds_example.ds.output_graph) > 0
        timestamp(fmt, "OK!")
    except Exception as err:
        timestamp(fmt, "ERROR")
        timestamp(fmt, err)
        raise Exception("fmt failed") from err


[d.on_click(_run_test) for d in tests]

# Show the test application

In [ ]:
W.VBox([ds_example, W.HBox([W.Label("Start Test"), *tests])])

## (Maybe) Run the test automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()